In [3]:
for i in range(10):
    ser.write(b'hello world')

In [4]:
ser.write(b'hello world')

11

# Setup

In [1]:
import time
import serial
ser = serial.Serial(port='/dev/serial0',baudrate=9600,parity=serial.PARITY_NONE,stopbits=serial.STOPBITS_ONE,bytesize=serial.EIGHTBITS,timeout=1)

In [3]:
from enum import Enum

VERBOSITY_QUIET = 0
VERBOSITY_DEBUG = 1
Global_Verbosity = VERBOSITY_QUIET


def set_Verbosity(verbosity):
    Global_Verbosity = verbosity


def printLog(str, verbosity):
    if verbosity <= Global_Verbosity:
        print(str)

class eMsgType(Enum):
    MODE_CONTROL = 0x10 #mode control
    COORDINATE = 0x20 #enemy coordinate
    ACK_BIT = 0x40 #always respond with the bit + this value of 40


class eSepChar(Enum): #start and ending hexes, acknowledgement bit
    STX_CHAR = 0x02
    ETX_CHAR = 0x03
    ACK_ASCII = b'ACK' 


class eAutoAimState(Enum):
    AUTO_AIM_INIT = 0 #turn on
    AUTO_AIM_IDLE = 1 #wait
    AUTO_AIM_SEND_COORDINATE = 2
    AUTO_AIM_WAIT_FOR_ACK = 3
    AUTO_AIM_SLEEP = 4 #turn off

class eReaderState(Enum):
    READ_STX = 0
    READ_PAYLOAD = 1
    READ_ERROR = 2


In [16]:
def AutoAim_Heartbeat(x,y,verbosity):
	# setup static variable
    #this is cv, arduino is control
	try: #is this backwards? i need to review try except statements
		printLog(AutoAim_Heartbeat.Heartbeat_State,VERBOSITY_DEBUG)
		printLog(AutoAim_Heartbeat.Reader_State,VERBOSITY_DEBUG)
	except:
		# main state, the state of the autoaim
		AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_INIT
		# sub state, your reader state
		AutoAim_Heartbeat.Reader_State = eReaderState.READ_STX
	
	if AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_INIT:
		ser.reset_input_buffer()
		ser.reset_output_buffer()
        ser.write(eSepChar.STX_CHAR + eMsgType.MODE_CONTROL + eSepChar.ACK_ASCII + eSepChar.ETX_CHAR ) 
        #send acknowledgement saying my mode has changed
		print("Reactor online. Sensors online. Weapons online. All systems nominal.")
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_IDLE:#if in idle mode
		# polling for control msg, if auto aim mode start msg received, ACK back
        
        #where should the no consecutive timer be implemented? 
		if ser.inWaiting()>0:
			if AutoAim_Heartbeat.Reader_State == eReaderState.READ_STX: #if you are reading STX
				if ser.read(1) == eSepChar.STX_CHAR.value: #check if the first bit is the STX value
					AutoAim_Heartbeat.Reader_State = eReaderState.READ_PAYLOAD	#if yes, enter reading state
			elif AutoAim_Heartbeat.Reader_State == eReaderState.READ_PAYLOAD: #if in reading state
				if ser.read(1) == eMsgType.MODE_CONTROL.value: #check if the bit you are recieving is telling you to change mode
                    #does ser.read read the first bit, or 1 of the next bit? fix thi
                    #read number probably wrong here, update it
                    #read the next bit
                    newState = ser.read(1)
                    match newState: #switch case, swap based on the bit that was read
                        case eAutoAimState.AUTO_AIM_INIT:
                            AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_INIT
                            print("I am changing my mode to initialization")
                        case eAutoAimState.AUTO_AIM_IDLE:
                            AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_IDLE
                            print("I am changing my mode to idle")
                        case eAutoAimState.AUTO_AIM_SEND_COORDINATE:
                            AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_SEND_COORDINATE
                            print("I am changing my mode to send coordinates")
                        case eAutoAimState.AUTO_AIM_WAIT_FOR_ACK:
                            AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_WAIT_FOR_ACK
                            print("I am changing my mode to waiting for acknowledgement")
                        case eAutoAimState.AUTO_AIM_SLEEP:
                            AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_SLEEP
                            print("I am changing my mode to waiting for sleep")
                    #send acknowledgement
                    ser.write(eSepChar.STX_CHAR + eMsgType.MODE_CONTROL + eSepChar.ACK_ASCII + eSepChar.ETX_CHAR ) 
				else:
					AutoAim_Heartbeat.Reader_State = eReaderState.READ_ERROR #read something wrong, throw error
			else:
				printLog("Unexpected AutoAim API state!")
				raise RuntimeError
		cc = ser.read(ser.inWaiting())
		print(cc.decode())
		print(cc)
        
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_SEND_COORDINATE:
        #CV to Control: Enemy coordinate 
        ser.write(eSepChar.STX_CHAR + eMsgType.COORDINATE + x + y + eSepChar.ETX_CHAR ) #the x + y might not be the correct form
        print("sending coordinates: " + x + ", " + y)
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_WAIT_FOR_ACK:
        
        print("waiting for acknowledgement from control.") #Idk what to do here
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_SLEEP:
	    #send acknowledgement
        ser.write(eSepChar.STX_CHAR + eMsgType.MODE_CONTROL + eSepChar.ACK_ASCII + eSepChar.ETX_CHAR ) 
        print("Auto aim disabled.")
    else:
		printLog("Unexpected AutoAim API state!")
		raise RuntimeError


	# CV should turn off if state is not active to save battery
	fCvActive = not (AutoAim_Heartbeat.Heartbeat_State in (eAutoAimState.AUTO_AIM_IDLE,eAutoAimState.AUTO_AIM_SLEEP))
	return fCvActive

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-16-af960430d055>, line 16)

In [ ]:
bytes(0x02).decode()

## Unit test

In [ ]:
AutoAim_Heartbeat()